In [1]:
#Se importan librerias
#numpy para operar con los arrays, cargar los resultados, etc...
import numpy as np
#matplotlib para graficar en 2D
import matplotlib.pyplot as plt
#pyvista para graficar en 3D (imagenes y videos)
import pyvista as pv
#curve_fit de scipy para trazar curvas que ajusten los datos
from scipy.optimize import curve_fit

In [3]:


#Informacion del mallado (PREPROCESSING)

with open('/home/juan/Documents/Estudiantes/2023/Mateo/Simulacion_disco/Pre-processing/gmsh/disco.msh','r') as m:
    #Se saltean las primeras 4 lineas
    for i in range(4):
        m.readline()
    #Se obtiene la cantidad de Physical Groups
    nPhysicalGroups = int(m.readline())
    #Se crea un diccionario vacio de los Physical Groups
    physicalGroups = {}
    #Se recorren los Physical Groups
    for i in range(nPhysicalGroups):
        #Se lee y se separa cada linea
        pg = m.readline().split()
        #Se convierte el numero identificador del Physical Group (str) a int
        idPhysGroup = int(pg[1])
        #Se asocia cada numero identificador a su respectivo nombre
        physicalGroups[idPhysGroup] = pg[2].replace('"', '')
    #Se saltean 2 lineas
    m.readline()
    m.readline()
    #Numero de nodos
    nNodos = int(m.readline())
    #Se crea ahora una matriz de coordenadas globales de los nodos de la forma
    # [X] = [[1,x1,y1,z1],
    #        [2,x2,y2,z2],
    #        [3,x3,y3,z3],
    #        [    ...   ],
    #        [n,xn,yn,zn]]
    X = np.loadtxt(m,max_rows=nNodos)
    #Se saltean 2 lineas
    m.readline()
    m.readline()
    #Numero de elementos totales
    nElementos = int(m.readline())
    #Numero de elementos triangulares (sale de statistics en gmsh)
    nElementos2D = 1332 #triangulos (de 3 nodos)
    #Numero de elementos tetraedricos (sale de statistics en gmsh)
    nElementos3D = 2272 #tetraedros (de 4 nodos)
    #Se almacena la informacion de los elementos en un array (conectividad)
    MC2D = np.loadtxt(m,dtype=int,max_rows=nElementos2D)
    MC3D = np.loadtxt(m,dtype=int,max_rows=nElementos3D)
    #Estos arrays contienen la informacion siguiente:
    #Columna 0: numero de elemento
    #Columna 1: numero del tipo de elemento (2: triangulo de 3 nodos, 
    #4: tetraedro de 4 nodos)
    #Columna 2: numero de ctes (siempre son 2, no es importante)
    #Columna 3: numero de physical group
    #Columna 4: numero de la superficie/volumen a la cual pertenece el 
    #elemento (no es importante)
    #Columna 5+: numeros de los nodos que pertenecen al elemento

#Se crea una matriz que contenga unicamente los nodos de cada elemento 3D
#Es decir, se filtra la matriz MC3D, de manera que queden unicamente las 
#ultimas 4 columnas
MC3D = np.delete(MC3D,[0,1,2,3,4],axis=1)

#Velocidad considerada (depende del caso)
V = 250 #km/h 
#Se cargan los resultados obtenidos del processing
#
path = f'/home/juan/Documents/Estudiantes/2023/Mateo/Simulacion_disco/Resultados/Análisis no lineal/{V}km_h/resultados_{V}.txt'
T = np.loadtxt(path,delimiter=',')
#Temperatura minima alcanzada
Tmin = np.amin(T)
#Temperatura maxima alcanzada
Tmax = np.amax(T)
print(f'Temperatura maxima: {Tmax}')


Temperatura maxima: 568.0980029302133


In [6]:
#IMAGENES DE CORTES
#Grafico auxiliar para sacar la colorbar
p = pv.Plotter(window_size=(250, 600),off_screen=True)
#Objeto de la malla inicial
msh = pv.read('/home/juan/Documents/Estudiantes/2023/Mateo/Simulacion_disco/Pre-processing/gmsh/disco.msh')
#Seteo de la leyenda
sargs = dict(width=0.2,height=0.8, vertical=True, position_x=0.5, position_y=0.1,title_font_size=14,label_font_size=16,font_family="courier")
#Asignacion de las temperaturas
msh.point_data['Temperatura [K]'] = Resultados[25] #t = 2.5 s
p.add_mesh(msh,scalars='Temperatura [K]',opacity=0,show_edges=False,clim=[T0,Tmax],cmap='autumn_r',scalar_bar_args=sargs)
p.show()
p.image_scale = 2
p.screenshot(f'colorbar_{Vkmh}.png')

#Objeto de la malla inicial
msh = pv.read('/home/juan/Documents/Estudiantes/2023/Mateo/Simulacion_disco/Pre-processing/gmsh/disco.msh')
#Inicio del graficador
p = pv.Plotter(shape=(2, 2),window_size=(500, 700),off_screen=True)

#1er grafico
#Seteo de la camara
camera = pv.Camera()
#Posicion de camara para corte
camera.position = (-0.6, 0, 0)
camera.focal_point = (0, 0, 0)
#Asignacion de la camara
p.camera = camera
#Asignacion de las temperaturas
msh.point_data['Temperatura [K]'] = T[50] #t = 5 s (se puede elegir cualquier tiempo)
#Corte vertical
corte_vertical = msh.slice(normal=[-1, 0, 0])
#corte_vertical.translate((0, -0.025, 0), inplace=True)
#Agregado de la malla en corte
corte = p.add_mesh(corte_vertical,scalars='Temperatura [K]',show_edges=False,clim=[T0,Tmax],cmap='autumn_r',scalar_bar_args=sargs)
p.remove_scalar_bar(title=None, render=True)

#2do grafico
p.subplot(0,1)
#Seteo de la camara
camera2 = pv.Camera()
#Posicion de camara para corte
camera2.position = (-0.6, 0, 0)
camera2.focal_point = (0, 0, 0)
camera2.elevation += 90
#Asignacion de la camara
p.camera = camera2
#Asignacion de las temperaturas
msh.point_data['Temperatura [K]'] = T[50] #t = 5 s (se puede elegir cualquier tiempo)
#Corte horizontal
corte_horizontal = msh.slice([0,1,0])
corte_horizontal.translate((0, 0, -0.05), inplace=True)
#Agregado de la malla en corte
corte = p.add_mesh(corte_horizontal,scalars='Temperatura [K]',show_edges=False,clim=[T0,Tmax],cmap='autumn_r')
p.remove_scalar_bar(title=None, render=True)

#3er grafico
p.subplot(1,0)
#Seteo de la camara
camera = pv.Camera()
#Posicion de camara para corte
camera.position = (-0.6, 0, 0)
camera.focal_point = (0, 0, 0)
#Asignacion de la camara
p.camera = camera
#Asignacion de las temperaturas
msh.point_data['Temperatura [K]'] = T[-1] #t =  tf (se puede elegir cualquier tiempo)
#Corte vertical
corte_vertical = msh.slice(normal=[-1, 0, 0])
#corte_vertical.translate((0, -0.025, 0), inplace=True)
#Agregado de la malla en corte
corte = p.add_mesh(corte_vertical,scalars='Temperatura [K]',show_edges=False,clim=[T0,Tmax],cmap='autumn_r',scalar_bar_args=sargs)
p.remove_scalar_bar(title=None, render=True)

#4to grafico
p.subplot(1,1)
#Seteo de la camara
camera2 = pv.Camera()
#Posicion de camara para corte
camera2.position = (-0.6, 0, 0)
camera2.focal_point = (0, 0, 0)
camera2.elevation += 90
#Asignacion de la camara
p.camera = camera2
#Asignacion de las temperaturas
msh.point_data['Temperatura [K]'] = T[-1] #t =  tf (se puede elegir cualquier tiempo)
#Corte horizontal
corte_horizontal = msh.slice([0,1,0])
corte_horizontal.translate((0, 0, -0.05), inplace=True)
#Agregado de la malla en corte
corte = p.add_mesh(corte_horizontal,scalars='Temperatura [K]',show_edges=False,clim=[T0,Tmax],cmap='autumn_r')
p.remove_scalar_bar(title=None, render=True)

#Se muestra la imagen
p.show()
#Se escala la imagen
p.image_scale = 2
#Se guarda la imagen
p.screenshot(f'corte_{Vkmh}.png')  


NameError: name 'Resultados' is not defined